# Astropy Tutorial

This tutorial will show how to detect object in an image using SEP. It will also show how to perform basic aperture photometry.

#### Import neccessary packages. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import sep                         #the package we will use to detect object
import astropy                     #imports astropy if you don't have it

from astropy.io import fits        #opens and works with the fits files
from matplotlib import rcParams    #parameters in matplotlib

rcParams['figure.figsize'] = [12., 10.]

save_path = 'C://Users/eyune/Desktop/' #creates a path for the saved images
                                       #change this be your desktop path

#### Open the FITS file.

Print out information from the file.

Note: fname is the path of the file on your laptop. Make sure to change fname to the correct path when implementing this tutorial.

In [ ]:
fname = "C://Users/eyune/Downloads/image.fits"     #path of file on my laptp
hdu_list = fits.open(fname)                        #opens the fits file
hdu_list.info()                                    #outputs the information of the file

# location of the image in the lsit and reassign this item to image_data
image_data = hdu_list[0].data

# print out the type of data the file is and the shape of the image
print("Type: ", type(image_data))
print("Dimensions: ", image_data.shape)

# close the fits file since it is now stored in image_data
hdu_list.close()

#Note: If WARNINGS arise, do not pay attention. This is just a formatting error in the fits file.

#### Use 'getdata()' to read in the image data and close the file and display the image.
Here we will obtain the data from the fits file and show the image using imshow() and a color map.

Numpy.mean() will compute the arithmetic mean along the specified axis.
Numpy.std() computes the standard deviation on the specified axis.

In [ ]:
image_data = fits.getdata(fname)    #obtains the actual data from the fits file

#show image
m, s = np.mean(image_data), np.std(image_data) 
plt.imshow(image_data, interpolation = 'nearest', cmap = 'gray', vmin = m-s, vmax = m+s,  origin = 'lower')
plt.colorbar()

#we want to save the background using plt.savefig() at a high resolution so that it looks nice.
#bbox_inches = 'tight' makes sure these images gets saved the same size and exactly how we see them in the notbook.
plt.savefig(save_path + 'raw_image.png', dpi = 470, bbox_inches = 'tight')

## Background Subtraction
Since there is a lot of noise from the background of the image, we must subtract the background from the optical/IR data. We don't want insiginificant brightness fluctuations to effect the object detection, so one way to fix that is to just subtract it out from the image.

#### Measure a spatially varying background on the image. 
Find the global mean and the noise of the background image so that we can eventually subtract it out of the image.

In [ ]:
#measuring a spatially  varying background, this will find the insignificant fluctuations
bkg = sep.Background(image_data, bw = 64, bh = 64, fw = 3, fh = 3)

# get a "global" mean and noise of the image background
print(bkg.globalback)
print(bkg.globalrms)

#### Evaluate the backgroud as 2-D array and show the background.

In [ ]:
#evaluate background as 2-d array, should be the same size as the original image
bkg_image = bkg.back()

#show the backgroun
plt.imshow(bkg_image, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar();

#we want to save the background using plt.savefig() at a high resolution so that it looks nice.
#bbox_inches = 'tight' makes sure these images gets saved the same size and exactly how we see them in the notbook.
plt.savefig(save_path + 'background.png', dpi = 470, bbox_inches = 'tight')

#### Evaluate the abckground noise as 2-D array and show the background noise.

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

#show the background noise
plt.imshow(bkg_rms, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar();

#we want to save the background using plt.savefig() at a high resolution so that it looks nice.
#bbox_inches = 'tight' makes sure these images gets saved the same size and exactly how we see them in the notbook.
plt.savefig(save_path + 'background_noise.png', dpi = 470, bbox_inches = 'tight')

#### Subtract the background from the image to obtain an image that is easier to detect objects from. 
This will give us the image without the background of fluctuations.

In [ ]:
data_sub = image_data - bkg

## Object Detection
We can perform the object detection from the background-subtracted image. Since the noise level is fairly flate, we set a detection threshold of 1.5σ. σ us the global background RMS.

#### sep.extract() will point out all the places with a detection threshold of 1.5σ. 

In [ ]:
objects = sep.extract(data_sub, 1.5, err = bkg.globalrms)

#Number of objects detected
len(objects)

#### Here we will plot the image and point out the objects. 
From here we will also know the object coordinates with other basic shape parameters.

In [ ]:
from matplotlib.patches import Ellipse    #Ellipse fitting

#plot bagkround_subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation = 'nearest', cmap = 'gray', vmin = m - s, vmax = m + s, origin = 'lower')

#plot an ellipse for each object
#objects['x'] and objects['y'] will give the centroid coordinates of the objects.
for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]), width = 6*objects['a'][i], height = 6*objects['b'][i], angle = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')   #leaves the center of the ellipse clear
    e.set_edgecolor('red')    #outlines the ellipse with a specified color
    ax.add_artist(e)          #add the ellipse to the image

#we want to save the background using plt.savefig() at a high resolution so that it looks nice.
#bbox_inches = 'tight' makes sure these images gets saved the same size and exactly how we see them in the notbook.
plt.savefig(save_path + 'image_object_detection.png', dpi = 470, bbox_inches = 'tight')

#### You can output more information from the 'objects' function. 
Here is a list of all the available field.

In [ ]:
objects.dtype.names

## Aperature Photometry
Now we will perform simple circular aperature photometery with a 3-pixel radius at the locations of the objects. Photometry is a technique used in astronomy to measure the intensity (flux) of an astronomical object electromagnetic radiation. So here, we will outputthe fluc of the objects.

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err = bkg.globalrms, gain = 1.0)

In [ ]:
#here we loop through each object to obtain its flux with error
for i in range(len(objects)):
    print("object {:d}: flux = {:f} +/- {:f}".format(i + 1, flux[i], fluxerr[i]))